# Proyecto Intermedio:  Parte 1

## Localización de señales $F_{1}$ y $F_{2}$

In [1]:
%matplotlib inline
#Para leer los archivos de matlab
from scipy.io import loadmat,whosmat
#Para fft
from numpy.fft import fft,fftshift,ifftshift
#Para calculo numérico
from numpy import arange,cos,abs,angle,pi,floor,array,where
# Para graficar
import matplotlib.pyplot as plt

## Importando los archivos de matlab a variables manejables en python

In [2]:
#Con este comando sabemos el identificador de la variable, su dimensión y su tipo
whosmat("../Archive/2_Datos.mat")

[('Data', (1, 5760000), 'double'), ('fs', (1, 1), 'double')]

In [3]:
archivo = loadmat("../Archive/2_Datos.mat")
data = array(archivo["Data"])
senal = data[0]
fs =  float(archivo["fs"])

### Frecuencia de muestreo

In [4]:
print(f"fs = {fs}")

fs = 96000.0


## Aplicando fft a la señal de entrada

In [5]:
fft_senal = fft(senal)

In [6]:
# Importando las subrutinas realizadas en clase para manejo de la fft
from calcVectFrec import calcVecFrec
from calculaVerdaderaAmplitud import calculaVerdaderaAmplitud
amplitud_fft = calculaVerdaderaAmplitud(fft_senal)
vfrec = calcVecFrec(senal,1/fs)

## Primer maximo

In [7]:
indice = where(max(amplitud_fft) == amplitud_fft)
print(indice)

(array([500000]),)


In [8]:
print(vfrec[indice])
f1 = vfrec[indice]

[8333.33333333]


## Segundo máximo

In [9]:
tol_frec = 10 # comenzara a buscar en +-10 en espacio de frecuencia luego del maximo global
lim_sup_vfreq = vfrec[indice] + tol_frec
lim_inf_vfreq = vfrec[indice] - tol_frec
ind_sup = where(vfrec == lim_sup_vfreq)
ind_inf = where(vfrec == lim_inf_vfreq)
ind_sup = int(ind_sup[0])
ind_inf = int(ind_inf[0])
print(vfrec[ind_sup],vfrec[ind_inf])
new_ind_max_sup = where(amplitud_fft[ind_sup:]==max(amplitud_fft[ind_sup:]))
new_ind_max_inf = where(amplitud_fft[:ind_inf] == max(amplitud_fft[:ind_inf]))
new_ind_max_sup = new_ind_max_sup[0] + ind_sup
new_ind_max_inf = new_ind_max_inf[0]
print(vfrec[new_ind_max_sup],amplitud_fft[new_ind_max_sup])
print(vfrec[new_ind_max_inf],amplitud_fft[new_ind_max_inf])
if( amplitud_fft[new_ind_max_sup] > amplitud_fft[new_ind_max_inf] ):
    f2 = vfrec[new_ind_max_sup]
else:
    f2 = vfrec[new_ind_max_inf]
if(f1 > f2):
    F2 = f1
    F1 = f2
else:
    F2 = f2
    F1 = f1

8343.333333333334 8323.333333333334
[10000.] [0.00468797]
[120.] [0.00330773]


In [10]:
print(f"Frecuencias encontradas: \n F1 = {F1}\n F2 = {F2}")

Frecuencias encontradas: 
 F1 = [8333.33333333]
 F2 = [10000.]


In [14]:
#Bucando las frecuencias dominantes del espectro
def buscar_frecuencias(senal,vfrecpos,tol):
    #Buscando maximo globales
    indice_max_global = where(senal == senal.max())
    f_max_global = vfrecpos[indice_max_global]
    #Definiendo un rango de tolerancia para comenzar a buscar otro maximo 
    # Buscaremos despues de f_max_global + tol y antes de f_max_global - tol
    tolerancia_frec = tol
    lim_sup_frec = f_max_global + tol
    lim_inf_frec = f_max_global - tol
    # Encontrando los indices del arrego de vfrecpos correspondientes a las frecuencias anteriores
    indice_lim_sup = where(vfrecpos == lim_sup_frec)
    indice_lim_inf = where(vfrecpos == lim_inf_frec)
    # Convirtiendo los valores a numeros enteros para poder realizar slicing
    indice_lim_sup = int(indice_lim_sup[0])
    indice_lim_inf = int(indice_lim_inf[0])
    # Encontramos el máximo de la región con indices 0:indice_lim_inf
    indice_max_inf = where(senal[:indice_lim_inf] == max(senal[:indice_lim_inf]))
    # y de la región con indices indice_lim_sup:end
    indice_max_sup = where(senal[indice_lim_sup:] == max(senal[indice_lim_sup:]))
    # note que a esta última región le tenemos que sumar el numero de indices recorridos ya
    indice_max_sup = indice_max_sup[0] + indice_lim_sup
    indice_max_inf = indice_max_inf[0]
    # Ahora decidiremos cual es el valor máximo entre las regiones inferior y superior establecidas
    # Encontramos el maximo relativo
    if(senal[indice_max_sup] > senal[indice_max_inf]):
        f_max_rel = vfrecpos[indice_max_sup]
    else:
        f_max_rel = vfrecpos[indice_max_inf]
    # Finalmente decidimos que frecuencia es mas alta y mas baja para encontrar asi f1 y f2
    if(f_max_global < f_max_rel):
        f_1 = f_max_global
        f_2 = f_max_rel
    else:
        f_1 = f_max_rel
        f_2 = f_max_global
    return f_1,f_2

In [15]:
n = len(fft_senal)
if(n%2 == 0):
    lim = floor(n/2)
else:
    lim = floor(n/2) + 1
vfrecpos = vfrec[0:int(lim)]

In [16]:
print(buscar_frecuencias(amplitud_fft,vfrecpos,10))

(array([8333.33333333]), array([10000.]))
